In [28]:
# import dependencies
import pandas as pd
import gmaps
import requests
import json
# Import API key
from config import g_key

In [2]:
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Amga,RU,2021-04-05 17:29:09,60.8953,131.9608,-6.59,99,100,2.08
1,1,Torbay,CA,2021-04-05 17:28:43,47.6666,-52.7314,42.01,100,90,13.80
2,2,Khatanga,RU,2021-04-05 17:27:08,71.9667,102.5000,4.71,85,79,9.73
3,3,Shiraz,IR,2021-04-05 17:30:26,29.6036,52.5388,62.60,15,0,6.91
4,4,Geraldton,AU,2021-04-05 17:28:19,-28.7667,114.6000,62.60,88,0,3.44


In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
temps = []
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of Humidity
# Get the latitude and longitude.

locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
humidity = city_data_df["Humidity"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heat map of percent cloudiness

# Get the latitude and longitude.

locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
clouds = city_data_df["Cloudiness"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=clouds)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Heat map of wind speed

# Get the latitude and longitude.

locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
wind = city_data_df["Wind Speed"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=wind)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [49]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) &\
                                       (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df = preferred_cities_df.dropna()


In [50]:
preferred_cities_df.count()

City_ID       182
City          182
Country       182
Date          182
Lat           182
Lng           182
Max Temp      182
Humidity      182
Cloudiness    182
Wind Speed    182
dtype: int64

In [51]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [53]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Carnarvon,AU,75.20,-24.8667,113.6333,
11,Mahebourg,MU,80.60,-20.4081,57.7000,
17,Pitimbu,BR,86.00,-7.4706,-34.8086,
21,Rikitea,PF,78.24,-23.1203,-134.9692,
22,Banjar,ID,82.40,-8.1900,114.9675,
25,Souillac,MU,80.60,-20.5167,57.5167,
28,Georgetown,MY,84.20,5.4112,100.3354,
29,Yenagoa,NG,82.31,4.9247,6.2642,
32,Maceio,BR,84.20,-9.6658,-35.7353,
33,Hithadhoo,MV,84.07,-0.6000,73.0833,


In [54]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [55]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Carnarvon,AU,75.20,-24.8667,113.6333,Hospitality Carnarvon
11,Mahebourg,MU,80.60,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
17,Pitimbu,BR,86.00,-7.4706,-34.8086,Reserva do Abiaí
21,Rikitea,PF,78.24,-23.1203,-134.9692,Pension Maro'i
22,Banjar,ID,82.40,-8.1900,114.9675,The Hamsa Resort


In [60]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [62]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))